In [59]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [66]:
from PIL import Image
import os

class DeepFakeDataset(Dataset):
    def __init__(self, manipulated_path, original_path, is_train):
        super().__init__()
        if (is_train):
            self.left_bound = 0
            self.right_bound = 0.8
        else:
            self.left_bound = 0.8
            self.right_bound = 1
        self.manipulated_path = manipulated_path
        self.manipulated_path_files = os.listdir(self.manipulated_path)
        self.manipulated_path_files = self.manipulated_path_files[int(len(self.manipulated_path_files) * self.left_bound):int(len(self.manipulated_path_files) * self.right_bound)]
        self.original_path = original_path
        self.original_path_files = os.listdir(self.original_path)
        self.original_path_files = self.original_path_files[int(len(self.original_path_files) * self.left_bound):int(len(self.original_path_files) * self.right_bound)]
    def __len__(self):
        return len(self.manipulated_path_files) + len(self.original_path_files)
    def __getitem__(self, idx):
        if (idx < len(self.manipulated_path_files)):
            return np.array(Image.open(f'{self.manipulated_path}/{self.manipulated_path_files[idx]}')), np.array([1, 0])
        else:
            return np.array(Image.open(f'{self.original_path}/{self.original_path_files[idx - len(self.manipulated_path_files)]}')), np.array([0, 1])

train_data = DeepFakeDataset('./data/manipulated', './data/original', True)
test_data = DeepFakeDataset('./data/manipulated', './data/original', False)
print(len(train_data))
print(len(test_data))
train_dataloader = DataLoader(train_data, batch_size=5, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=5, shuffle=True)
for batch in train_dataloader:
    print(batch[0].shape, batch[1].shape)
    break

9600
2400
torch.Size([5, 299, 299, 3]) torch.Size([5, 2])


In [67]:
class BenchmarkCNN(nn.Module):
    def __init__(self, input_shape, num_classes):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(128 * 14 * 14, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes),
            nn.Softmax(dim=1),
        ])
        '''x = Input(shape=input_shape)
        h = Conv2D(32, padding='same', kernel_size=(3,3), activation='relu')(x)
        h = Conv2D(32, padding='same', kernel_size=(3,3), activation='relu')(x)
        h = MaxPooling2D(pool_size=(2,2))(h)
        h = Conv2D(64, padding='same', kernel_size=(3,3), activation='relu')(h)
        h = Conv2D(64, padding='same', kernel_size=(3,3), activation='relu')(h)
        h = MaxPooling2D(pool_size=(2,2))(h)
        h = Conv2D(128, kernel_size=(3,3), activation='relu')(h)
        h = Conv2D(128, kernel_size=(3,3), activation='relu')(h)
        h = MaxPooling2D(pool_size=(2,2))(h)
        h = Flatten()(h)
        h = Dense(128, activation='relu')(h)
        h = Dropout(.5)(h)
        output = Dense(target_labels.shape[0], activation='softmax')(h)

        model = tf.keras.Model(inputs=x, outputs=output)

        model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])'''

    def forward(self, X):
        for module in self.layers:
            # print("------")
            # print(X.shape)
            # print(module)
            X = module(X)
        return self.layers

In [68]:
model = BenchmarkCNN((128, 128, 3), 2)

In [69]:
import numpy as np

x = np.zeros((1, 3, 128, 128), dtype=np.float32)
pred = model(torch.tensor(x))

In [ ]:
# https://zhuanlan.zhihu.com/p/434477086